In [1]:
import pandas as pd 
import numpy as np
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
import itertools

Using TensorFlow backend.


## Dataset preparation

In [0]:
def shuffle(df, n=1, axis=0):  
    df = df.copy()
    for _ in range():
        df.apply(np.random.shuffle, axis=axis)
    return df

In [3]:
df_bjp = pd.read_csv('bjp_data.csv',encoding="ISO-8859-1")
df_bjp = df_bjp.rename(columns = {'commentText': 'SentimentText', 'Label': 'Sentiment'})
df_bjp.head()

,SentimentText,Sentiment
0,Banda apna 100 % best he,1
1,WE NEED THE KING MAKER ---- NARENDRA MODI,1
2,Awesome... Banda apna Best hai,1
3,Best PM ever,1
4,RAHUL GANDHI LEFT THE CHAT,1


In [0]:
# df_cong = pd.read_csv('CONGRESS.csv',encoding="ISO-8859-1")
# df_cong = df_cong.rename(columns = {'ï»¿commentText': 'SentimentText', 'Label': 'Sentiment'})

## Data Preprocessing

In [0]:
import re
#handling links
df_bjp['SentimentText'] = df_bjp['SentimentText'].str.replace("https?://\S*", " ")
#handling user tags
df_bjp['SentimentText'] = df_bjp['SentimentText'].str.replace("@[\w]*", " ")
#handling special character,punctuation,numbers
df_bjp['SentimentText'] = df_bjp['SentimentText'].str.replace("[^a-zA-Z#]", " ")
#fix misspelled words
df_bjp['SentimentText'] = df_bjp['SentimentText'].apply(lambda x: ''.join(''.join(s)[:2] for _, s in itertools.groupby(x)))

df_bjp['SentimentText'] = df_bjp['SentimentText'].apply(lambda x: ' '.join(re.sub("(@[A-Za-z0-9]+)|(#[A-Za-z0-9]+)", " ", x).split()))

df_bjp['SentimentText'] = df_bjp['SentimentText'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))

In [6]:
df_bjp.head()

,SentimentText,Sentiment
0,Banda apna best,1
1,NEED THE KING MAKER NARENDRA MODI,1
2,Awesome Banda apna Best hai,1
3,Best ever,1
4,RAHUL GANDHI LEFT THE CHAT,1


In [7]:
import nltk
# nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
stopwords = []
with open('hinglishStopWords.txt', 'r') as f:
    hinglishStopWords = f.read().split('\n')
stopwords+=hinglishStopWords

with open('englishStopWords','r') as f:
    englishStopWords = f.read().split('\n')
stopwords+=englishStopWords

In [0]:
from nltk.tokenize import word_tokenize

def tokenization(tweet):
    text = []
    for token in word_tokenize(tweet):
        if token in stopwords:
            continue
        else:
            text.append(token)
    return text

In [0]:
df_bjp['SentimentText'] = df_bjp['SentimentText'].apply(lambda x: tokenization(x))

In [0]:
df_bjp['SentimentText'] = df_bjp['SentimentText'].apply(lambda x: ' '.join(x) )

In [0]:
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df_bjp['SentimentText'], df_bjp['Sentiment'])

In [43]:
count_vect = CountVectorizer(analyzer='word', tokenizer=tokenization)
count_vect.fit(df_bjp['SentimentText'])

xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(valid_x)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [30]:
classifier = naive_bayes.MultinomialNB()
# classifier = svm.SVC()
# classifier = xgboost.XGBClassifier()
classifier.fit(xtrain_count, train_y)
predictions = classifier.predict(xvalid_count)
print(predictions.shape)
metrics.accuracy_score(predictions, valid_y)

(488,)


0.8155737704918032

In [44]:
input_size = xtrain_count.shape[1]
input_layer = layers.Input((input_size, ),)
hidden_layer1 = layers.Dense(1024, activation="relu")(input_layer)
hidden_layer2 = layers.Dense(512, activation="relu")(hidden_layer1)
hidden_layer3 = layers.Dense(256, activation="relu")(hidden_layer2)
hidden_layer = layers.Dense(128, activation="relu")(hidden_layer3)
output_layer = layers.Dense(1, activation="sigmoid")(hidden_layer)
classifier = models.Model(inputs = input_layer, outputs = output_layer)
classifier.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
classifier.fit(xtrain_count, train_y,epochs=50)
predictions = classifier.predict(xvalid_count)

Epoch 1/50
1464/1464 [==============================] - 3s 2ms/step - loss: 0.5465
Epoch 2/50
1464/1464 [==============================] - 3s 2ms/step - loss: 0.1851
Epoch 3/50
1464/1464 [==============================] - 3s 2ms/step - loss: 0.0929
Epoch 4/50
1464/1464 [==============================] - 3s 2ms/step - loss: 0.0594
Epoch 5/50
1464/1464 [==============================] - 3s 2ms/step - loss: 0.0513
Epoch 6/50
1464/1464 [==============================] - 3s 2ms/step - loss: 0.0414
Epoch 7/50
1464/1464 [==============================] - 3s 2ms/step - loss: 0.0334
Epoch 8/50
1464/1464 [==============================] - 3s 2ms/step - loss: 0.0364
Epoch 9/50
1464/1464 [==============================] - 3s 2ms/step - loss: 0.0342
Epoch 10/50
1464/1464 [==============================] - 3s 2ms/step - loss: 0.0336
Epoch 11/50
1464/1464 [==============================] - 3s 2ms/step - loss: 0.0344
Epoch 12/50
1464/1464 [==============================] - 3s 2ms/step - loss: 0.0328
E

In [0]:
predictionf = predictions[:,0] >= 0.5

In [49]:
metrics.accuracy_score(predictionf, valid_y)

0.8381147540983607

In [50]:
from sklearn.metrics import f1_score
f1_score(valid_y, predictionf)

0.8454011741682974

### Tfidf ngram


In [51]:
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', tokenizer=tokenization, ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram.fit(df_bjp['SentimentText'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [55]:
# classifier = naive_bayes.MultinomialNB()
classifier = svm.SVC()
# classifier = xgboost.XGBClassifier()
classifier.fit(xtrain_tfidf_ngram, train_y)
predictions = classifier.predict(xvalid_tfidf_ngram)
print(predictions.shape)
metrics.accuracy_score(predictions, valid_y)

(488,)


0.7930327868852459

In [57]:
input_size = xtrain_tfidf_ngram.shape[1]
input_layer = layers.Input((input_size, ))
hidden_layer1 = layers.Dense(1024, activation="relu")(input_layer)
hidden_layer2 = layers.Dense(512, activation="relu")(hidden_layer1)
hidden_layer3 = layers.Dense(256, activation="relu")(hidden_layer2)
hidden_layer = layers.Dense(128, activation="relu")(hidden_layer3)
output_layer = layers.Dense(1, activation="sigmoid")(hidden_layer)
classifier = models.Model(inputs = input_layer, outputs = output_layer)
classifier.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
classifier.fit(xtrain_tfidf_ngram, train_y,epochs=50)
predictions = classifier.predict(xvalid_tfidf_ngram)

Epoch 1/50
1464/1464 [==============================] - 4s 3ms/step - loss: 0.6068
Epoch 2/50
1464/1464 [==============================] - 4s 2ms/step - loss: 0.2454
Epoch 3/50
1464/1464 [==============================] - 4s 2ms/step - loss: 0.1784
Epoch 4/50
1464/1464 [==============================] - 3s 2ms/step - loss: 0.1647
Epoch 5/50
1464/1464 [==============================] - 4s 2ms/step - loss: 0.1606
Epoch 6/50
1464/1464 [==============================] - 3s 2ms/step - loss: 0.1611
Epoch 7/50
1464/1464 [==============================] - 3s 2ms/step - loss: 0.1578
Epoch 8/50
1464/1464 [==============================] - 4s 2ms/step - loss: 0.1636
Epoch 9/50
1464/1464 [==============================] - 4s 2ms/step - loss: 0.1556
Epoch 10/50
1464/1464 [==============================] - 4s 2ms/step - loss: 0.1550
Epoch 11/50
1464/1464 [==============================] - 4s 2ms/step - loss: 0.1560
Epoch 12/50
1464/1464 [==============================] - 4s 2ms/step - loss: 0.1546
E

In [58]:
predictionf = predictions[:,0] >= 0.5
metrics.accuracy_score(predictionf, valid_y)

0.7745901639344263

In [59]:
from sklearn.metrics import f1_score
f1_score(valid_y, predictionf)

0.808362369337979